In [1]:
from dotenv import load_dotenv
load_dotenv()

True

[from](https://python.langchain.com/v0.2/docs/tutorials/agents/)

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
#search_results = search.invoke("what is the weather in SF")
#print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

In [3]:
from langchain_groq import ChatGroq

#model = ChatGroq(model="llama3-8b-8192")
#model = ChatGroq(model="llama3-70b-8192")
#model = ChatGroq(model="llama-3.1-70b-versatile")
model = ChatGroq(model="llama-3.1-8b-instant")

In [4]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content


"How's it going? Is there something I can help you with or would you like to chat?"

In [5]:
model_with_tools = model.bind_tools(tools)

In [8]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: <tool-use>
{
  "tool_calls": []
}
</tool-use>
ToolCalls: []


In [10]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather'}, 'id': 'call_79vj', 'type': 'tool_call'}]


In [9]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [11]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='3a25d86e-219b-4f07-a6d0-be1cc6632f37'),
 AIMessage(content='<tool-use>\n{"tool_calls": []}\n</tool-use>', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 941, 'total_tokens': 956, 'completion_time': 0.02, 'prompt_time': 0.208624881, 'queue_time': None, 'total_time': 0.228624881}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-2e959d45-7bac-4f57-a57e-f9d578088d38-0', usage_metadata={'input_tokens': 941, 'output_tokens': 15, 'total_tokens': 956})]

# Adding in memory
As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a thread_id when invoking the agent (so it knows which thread/conversation to resume from).

In [12]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [13]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='<tool-use>{"tool_calls":[]}</tool-use>', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 943, 'total_tokens': 956, 'completion_time': 0.017333333, 'prompt_time': 0.207916753, 'queue_time': None, 'total_time': 0.22525008600000002}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'stop', 'logprobs': None}, id='run-248bb5ff-232a-498b-b07a-13d0813a60a5-0', usage_metadata={'input_tokens': 943, 'output_tokens': 13, 'total_tokens': 956})]}}
----


In [15]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 991, 'total_tokens': 997, 'completion_time': 0.008, 'prompt_time': 0.219766424, 'queue_time': None, 'total_time': 0.227766424}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-ae0161c3-cfa5-4169-b107-2a79cf79c54d-0', usage_metadata={'input_tokens': 991, 'output_tokens': 6, 'total_tokens': 997})]}}
----


Example [LangSmith trace](https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)

If I want to start a new conversation, all I have to do is change the thread_id used

In [18]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='<tool-use>\n{\n  "tool_calls": []\n}\n</tool-use>', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 974, 'total_tokens': 991, 'completion_time': 0.022666667, 'prompt_time': 0.214857042, 'queue_time': None, 'total_time': 0.237523709}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'stop', 'logprobs': None}, id='run-cbe56da5-9eed-4487-a269-3a26c80c0cfe-0', usage_metadata={'input_tokens': 974, 'output_tokens': 17, 'total_tokens': 991})]}}
----


# Conclusion
That's a wrap! In this quick start we covered how to create a simple agent. We've then shown how to stream back a response - not only the intermediate steps, but also tokens! We've also added in memory so you can have a conversation with them. Agents are a complex topic, and there's lot to learn!

For more information on Agents, please check out the LangGraph documentation. This has it's own set of concepts, tutorials, and how-to guides.